In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import time

2023-05-30 19:16:37.424469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset, info = tfds.load('cifar10', shuffle_files=True, with_info=True, as_supervised=True)
train_dataset = dataset['train']
test_dataset = dataset['test']

train_dataset = train_dataset.map(lambda image, label: (tf.image.resize(image, (32, 32)), label))
test_dataset = test_dataset.map(lambda image, label: (tf.image.resize(image, (32, 32)), label))

train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32)

2023-05-30 19:16:39.367746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-30 19:16:39.423302: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-30 19:16:39.423792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10))

In [4]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [6]:
start_time = time.time()
with tf.device('/GPU:0'):
    model.fit(train_dataset, epochs=10, validation_data=test_dataset)
end_time = time.time()
gpu_training_time = end_time - start_time

# Оценка точности модели на GPU
with tf.device('/GPU:0'):
    test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print('Accuracy of the network on the test images (GPU):', test_acc)
print('Training time on GPU:', gpu_training_time)

Epoch 1/10


2023-05-30 19:16:46.238755: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-05-30 19:16:46.776305: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f4830028540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-30 19:16:46.776384: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2023-05-30 19:16:46.789694: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-30 19:16:46.892729: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1554/1563 [============================>.] - ETA: 0s - loss: 3.7331 - accuracy: 0.2110

2023-05-30 19:16:50.686456: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-05-30 19:16:50.686883: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]


1563/1563 [==============================] - 6s 2ms/step - loss: 3.7230 - accuracy: 0.2117 - val_loss: 2.0763 - val_accuracy: 0.2742
Epoch 2/10
1563/1563 [==============================] - 3s 2ms/step - loss: 2.0340 - accuracy: 0.2821 - val_loss: 2.0077 - val_accuracy: 0.2981
Epoch 3/10
1563/1563 [==============================] - 3s 2ms/step - loss: 1.9227 - accuracy: 0.3252 - val_loss: 1.9329 - val_accuracy: 0.3358
Epoch 4/10
1563/1563 [==============================] - 3s 2ms/step - loss: 1.8268 - accuracy: 0.3618 - val_loss: 2.2012 - val_accuracy: 0.3514
Epoch 5/10
1563/1563 [==============================] - 3s 2ms/step - loss: 1.7600 - accuracy: 0.3807 - val_loss: 2.6483 - val_accuracy: 0.3158
Epoch 6/10
1563/1563 [==============================] - 3s 2ms/step - loss: 1.7181 - accuracy: 0.3952 - val_loss: 2.4625 - val_accuracy: 0.3427
Epoch 7/10
1563/1563 [==============================] - 3s 2ms/step - loss: 1.6809 - accuracy: 0.4122 - val_loss: 2.5080 - val_accuracy: 0.3209
Epo